In [ ]:
import numpy as np 
from keras.datasets import mnist 
from keras.layers import (Activation, 
                         BatchNormalization,
                         Dense, Flatten, Dropout, Reshape)
from keras.layers.advanced_activations import LeakyReLU 
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Sequential 
from keras.optimizers import Adam 

In [ ]:
#Input Dimensions 
imgRows = 28
imgCols = 28
channels = 1
imgShape = (imgRows,imgCols,channels)
zDim = 100 #Input noise vector 

In [ ]:
def buildGenerator(zDim):
    model = Sequential()
    model.add(Dense(256*7*7,input_dim = zDim))
    model.add(Reshape(7,7,256))
    model.add(Conv2DTranspose(128,kernel_size = 3,strides = 2,padding = 'same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Conv2DTranspose(64,kernel_size = 3,strides = 1,padding = 'same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Conv2DTranspose(1,kernel_size = 3,strides = 2,padding = 'same'))
    model.add(Activation('tanh'))
    return model 

In [ ]:
def buildDiscriminator(imgShape):
    model = Sequential()
    model.add(Conv2D(32,kernel_size = 3,strides = 2,input_shape = imgShape,padding = 'same'))
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Conv2D(64,kernel_size = 3,strides = 2,input_shape = imgShape,padding = 'same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Conv2D(128,kernel_size = 3,strides = 2,input_shape = imgShape,padding = 'same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpah = 0.01))
    model.add(Flatten())
    model.add(Dense(1,activation = 'sigmoid'))
    return model 

In [1]:
def buildDCGAN(G,D):
    model = Sequential()
    model.add(G)
    model.add(D)
    return model

SyntaxError: unexpected EOF while parsing (<ipython-input-1-87580fa2b74e>, line 1)

In [ ]:
discriminator = buildDiscriminator(imgShape)
discriminator.compile(loss = 'binary_crossentropy',optimizer = Adam(),metrics = ['accuracy'])
generator = buildGenerator(zDim)
discriminator.trainable = False
DCGAN = buildDCGAN(generator,discriminator)
DCGAN.compile(loss = 'binary_crossentropy',optimizer = Adam())

In [ ]:
def sampleImages(G,imgGridRows = 4,imgGridCols = 4):
    z = np.random.normal(0,1,imgGridRows*imgGridCols*zDim)
    genImgs = G.predict(z)
    genImgs = 0.5*genImgs + 0.5
    fig,ax = plt.subplots(imgGridRows,imgGridCols,figsize = (4,4),sharey = True,sharex = True)
    cnt = 0
    for i in range(imgGridRows):
        for j in range(imgGridCols):
            axs[i,j].imshow(genImgs[cnt,:,:,0],cmap = 'gray')
            axs[i,j].axis('off')
            cnt += 1

In [ ]:
#The GAN training loop 
losses = []; accuracies = []
iterationCheckpoints = []
def train(iterations,batchSize,sampleInterval):
    (x_train,_),(_,_) = mnist.load_data()
    x_train = x_train/127.0 - 1.0 
    x_train = np.expand_dims(x_train,axis = 3)
    real = np.ones((batchSize,1))
    fake = np.zeros((batchSize,1))
    for iteration in range(iterations):
        idx = np.random.randint(0,x_train.shape[0],batchSize)
        imgs = x_train[idx]
        z = np.random.normal(0,1,(batchSize,100))
        genImgs = generator.predict(z)
        dLossReal = discriminator.train_on_batch(imgs,real)
        dLossFake = discriminator.train_on_batch(genImgs,fake)
        dLoss,accuracy = 0.5*np.add(dLossReal,dLossFake)
        z = np.random.normal(0,1,(batchSize,100))
        genImgs = generator.predict(z)
        gLoss = DCGAN.train_on_batch(z,real)
        if(iteration+1)%sampleInterval == 0:
            losses.append((dLoss,gLoss))
            accuracies.append(100.0*accuracy)
            iterationCheckpoints.append(iteration+1)
            sampleImages(generator)
            
            
        